# Descriptive analysis of the results

In [9]:
# Libraries
import numpy as np
import pandas as pd
import json

In [12]:
green_skills_normalized = pd.read_csv("../data/green_jobs_normalized.csv")
id_to_skill = json.load(open("../data/mapping/id_to_skill.json", "r"))

a, b = green_skills_normalized.shape
print(f"Number of green skills detected: {a}")
print(f"Number of unique green skills detected: {len(green_skills_normalized['skill_id'].unique())}")

Number of green skills detected: 5037
Number of unique green skills detected: 721


In [48]:
id_to_skill = json.load(open("../data/mapping/id_to_skill.json", "r"))
id_to_job = json.load(open("../data/mapping/id_to_job.json", "r"))

joined_data = green_skills_normalized.copy()

In [55]:
for row in joined_data.itertuples():
    skill_id = row.skill_id
    skill_name = id_to_skill.get(str(skill_id), "Unknown Skill")
    joined_data.at[row.Index, 'skill_name'] = skill_name

In [60]:
joined_data

,job_id,skill_id,similarity_score,skill_name
0,243ef92c3012f0f0,2012,0.537140,"[lean manufacturing, WCM world class manufactu..."
1,e2e422fd6ea193fb,211,0.508682,"[develop management plans, management plans pr..."
2,0dffb51a47828590,2424,0.506158,"[coordinate shipments of recycling materials, ..."
3,0dffb51a47828590,1577,0.502190,"[store sorted waste, sorted waste storing]"
4,0dffb51a47828590,1214,0.509662,"[inspect recycling procedures, recycling proce..."
...,...,...,...,...
5032,https://www.occ.com.mx/empleos/de-Stellantis/e...,1799,0.588585,"[develop food policy, launch food policy]"
5033,https://www.occ.com.mx/empleos/de-Stellantis/e...,2083,0.556318,"[develop recycling programs, launch recycling ..."
5034,https://www.occ.com.mx/empleos/de-Stellantis/e...,2013,0.565457,"[lean manufacturing, manufacturing excellence]"
5035,https://www.occ.com.mx/empleos/de-Stellantis/e...,156,0.502869,[interpret scientific data to assess water qua...


In [77]:
jobs = pd.read_csv("../data/jan_to_apr_2025_with_languages_cleaned.csv")
jobs.head(5)

,Title,Job_ID,source,Skills,month,detected_language
0,ventas flotillas,977bbdb05a29b771,indeed,experiencia en prospeccion de clientes,1.0,es
1,ventas flotillas,977bbdb05a29b771,indeed,experiencia en cierre de ventas,1.0,es
2,ventas flotillas,977bbdb05a29b771,indeed,experiencia en giro automotriz,1.0,es
3,ventas flotillas,977bbdb05a29b771,indeed,licencia de manejo indispensable,1.0,es
4,ventas flotillas,977bbdb05a29b771,indeed,habilidad para la prospeccion en empresas (tel...,1.0,es


In [79]:
joined_data

,job_id,skill_id,similarity_score,skill_name
0,243ef92c3012f0f0,2012,0.537140,"[lean manufacturing, WCM world class manufactu..."
1,e2e422fd6ea193fb,211,0.508682,"[develop management plans, management plans pr..."
2,0dffb51a47828590,2424,0.506158,"[coordinate shipments of recycling materials, ..."
3,0dffb51a47828590,1577,0.502190,"[store sorted waste, sorted waste storing]"
4,0dffb51a47828590,1214,0.509662,"[inspect recycling procedures, recycling proce..."
...,...,...,...,...
5032,https://www.occ.com.mx/empleos/de-Stellantis/e...,1799,0.588585,"[develop food policy, launch food policy]"
5033,https://www.occ.com.mx/empleos/de-Stellantis/e...,2083,0.556318,"[develop recycling programs, launch recycling ..."
5034,https://www.occ.com.mx/empleos/de-Stellantis/e...,2013,0.565457,"[lean manufacturing, manufacturing excellence]"
5035,https://www.occ.com.mx/empleos/de-Stellantis/e...,156,0.502869,[interpret scientific data to assess water qua...
